In [33]:
dir

<function dir>

In [34]:
pwd

'/Users/pichappan/Documents/SamplePython/input'

In [35]:
cd ../input

/Users/pichappan/Documents/SamplePython/input


In [36]:
import pandas as pd
import numpy as np
from sklearn import linear_model

In [37]:
dft = pd.read_csv('http://bit.ly/kaggletrain')
df1 = pd.read_csv("Airbnb_NYC.csv")
df1.tail()

,Boroughs,Latitude,Longitude,Prop_Type,Min_Nights,Host_Listing_Cnt,Days_Available,Review_Cnt,Reviews30d,Price
38728,Staten Island,40.64641,-74.08502,EntireHome,2.0,1.0,106,58,4.73,130.0
38729,Staten Island,40.64682,-74.08734,PrivateRoom,20.0,1.0,89,58,1.05,43.0
38730,Staten Island,40.64730,-74.08408,PrivateRoom,30.0,4.0,0,85,1.35,105.0
38731,Staten Island,40.64762,-74.08682,EntireHome,3.0,1.0,232,5,1.88,135.0
38732,Staten Island,40.64779,-74.08460,EntireHome,1.0,1.0,250,108,2.32,140.0


In [38]:
# Data Preprocessing
df1 = df1.dropna(thresh=9)
df1.Min_Nights.fillna(df1.Min_Nights.median())
df1.Host_Listing_Cnt.fillna(df1.Host_Listing_Cnt.median())
df1.Days_Available.fillna(df1.Days_Available.median())
df1.Review_Cnt.fillna(df1.Review_Cnt.median())
df1.Reviews30d.fillna(df1.Reviews30d.median())
df1.Price.fillna(df1.Price.median())
df1.Prop_Type.value_counts()

EntireHome     20173
PrivateRoom    17720
SharedRoom       840
Name: Prop_Type, dtype: int64

In [41]:
# New Column with map method is quick and easy
df1['isEntireHome'] = df1.Prop_Type.map({'EntireHome':1,'PrivateRoom':0,'SharedRoom':0})
df1['isPrivateRoom'] = df1.Prop_Type.map({'EntireHome':0,'PrivateRoom':1,'SharedRoom':0})
df1

,Boroughs,Latitude,Longitude,Prop_Type,Min_Nights,Host_Listing_Cnt,Days_Available,Review_Cnt,Reviews30d,Price,isEntireHome,isPrivateRoom
0,Bronx,40.80011,-73.91330,PrivateRoom,21.0,2.0,291,19,0.27,60.0,0,1
1,Bronx,40.80024,-73.91422,PrivateRoom,3.0,1.0,0,59,2.93,49.0,0,1
2,Bronx,40.80074,-73.91574,PrivateRoom,3.0,1.0,354,3,0.87,50.0,0,1
3,Bronx,40.80091,-73.91449,PrivateRoom,21.0,2.0,117,18,0.36,71.0,0,1
4,Bronx,40.80143,-73.91353,EntireHome,2.0,3.0,0,12,3.50,115.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
38728,Staten Island,40.64641,-74.08502,EntireHome,2.0,1.0,106,58,4.73,130.0,1,0
38729,Staten Island,40.64682,-74.08734,PrivateRoom,20.0,1.0,89,58,1.05,43.0,0,1
38730,Staten Island,40.64730,-74.08408,PrivateRoom,30.0,4.0,0,85,1.35,105.0,0,1
38731,Staten Island,40.64762,-74.08682,EntireHome,3.0,1.0,232,5,1.88,135.0,1,0


In [42]:
# One Hot Encoding , Dummy Variables. This is flexable 
dummyBoro = pd.get_dummies(df1.Boroughs, prefix='is').iloc[:,:4] # [all rows, 4 column from start] dropped last column
dummyProp = pd.get_dummies(df1.Prop_Type, prefix='is').iloc[:,1:] # [all rows, column 1 through end] dropped first column

In [43]:
# df2 = pd.concat([df1,dummyBoro, dummyProp], axis='columns') # concat 3 df
df2 = pd.concat([df1,dummyBoro], axis='columns') # or axis=1
df2.head()

,Boroughs,Latitude,Longitude,Prop_Type,Min_Nights,Host_Listing_Cnt,Days_Available,Review_Cnt,Reviews30d,Price,isEntireHome,isPrivateRoom,is_Bronx,is_Brooklyn,is_Manhattan,is_Queens
0,Bronx,40.80011,-73.91330,PrivateRoom,21.0,2.0,291,19,0.27,60.0,0,1,1,0,0,0
1,Bronx,40.80024,-73.91422,PrivateRoom,3.0,1.0,0,59,2.93,49.0,0,1,1,0,0,0
2,Bronx,40.80074,-73.91574,PrivateRoom,3.0,1.0,354,3,0.87,50.0,0,1,1,0,0,0
3,Bronx,40.80091,-73.91449,PrivateRoom,21.0,2.0,117,18,0.36,71.0,0,1,1,0,0,0
4,Bronx,40.80143,-73.91353,EntireHome,2.0,3.0,0,12,3.50,115.0,1,0,1,0,0,0


In [46]:
# Dummy variable Trap. Multi Colinear variables. Now of catagorical variables
df3 = df2.drop(['Boroughs','Prop_Type'],axis='columns')
df3.dropna(inplace=True) # Drop rows with NaN
df3.shape

(38331, 14)

In [48]:
# is all this too hard? don't worry we have a quick and easy way. Pass df instand of Series to pd.get_dummies
df2=pd.get_dummies(df1, columns=['Boroughs','Prop_Type'], drop_first=True)
df2

,Latitude,Longitude,Min_Nights,Host_Listing_Cnt,Days_Available,Review_Cnt,Reviews30d,Price,isEntireHome,isPrivateRoom,Boroughs_Brooklyn,Boroughs_Manhattan,Boroughs_Queens,Boroughs_Staten Island,Prop_Type_PrivateRoom,Prop_Type_SharedRoom
0,40.80011,-73.91330,21.0,2.0,291,19,0.27,60.0,0,1,0,0,0,0,1,0
1,40.80024,-73.91422,3.0,1.0,0,59,2.93,49.0,0,1,0,0,0,0,1,0
2,40.80074,-73.91574,3.0,1.0,354,3,0.87,50.0,0,1,0,0,0,0,1,0
3,40.80091,-73.91449,21.0,2.0,117,18,0.36,71.0,0,1,0,0,0,0,1,0
4,40.80143,-73.91353,2.0,3.0,0,12,3.50,115.0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38728,40.64641,-74.08502,2.0,1.0,106,58,4.73,130.0,1,0,0,0,0,1,0,0
38729,40.64682,-74.08734,20.0,1.0,89,58,1.05,43.0,0,1,0,0,0,1,1,0
38730,40.64730,-74.08408,30.0,4.0,0,85,1.35,105.0,0,1,0,0,0,1,1,0
38731,40.64762,-74.08682,3.0,1.0,232,5,1.88,135.0,1,0,0,0,0,1,0,0


In [ ]:
#Class Object
lrModel = linear_model.LinearRegression()

In [ ]:
X = df3.drop('Price',axis='columns')
X

In [ ]:
y = df3.Price
y

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
#same output when random_state
Xlearn, Xtest, yLearn, yTest = train_test_split(X, y, test_size=0.35, random_state=10)

In [ ]:
Xtest.shape

In [ ]:
Xtest

In [ ]:
Xlearn.shape

In [ ]:
# Training or Learning
lrModel.fit(Xlearn,yLearn)

In [ ]:
lrModel.predict([[40.80024, -73.91422, 13.0, 1.0, 0, 29, 2.93, 1, 0, 0, 0, 1,0]])

In [ ]:
lrModel.score(Xtest,yTest)

In [ ]:
lrModel.predict(Xtest)

In [ ]:
yTest